<a href="https://colab.research.google.com/github/PraneethEmandi/chatbot/blob/main/SIH_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade httpx


  Using cached httpx-0.26.0-py3-none-any.whl (75 kB)
  Using cached httpcore-1.0.2-py3-none-any.whl (76 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Attempting uninstall: h11
    Found existing installation: h11 0.9.0
    Uninstalling h11-0.9.0:
      Successfully uninstalled h11-0.9.0
  Attempting uninstall: httpcore
    Found existing installation: httpcore 0.9.1
    Uninstalling httpcore-0.9.1:
      Successfully uninstalled httpcore-0.9.1
  Attempting uninstall: httpx
    Found existing installation: httpx 0.13.3
    Uninstalling httpx-0.13.3:
      Successfully uninstalled httpx-0.13.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
googletrans 4.0.0rc1 requires httpx==0.13.3, but you have httpx 0.26.0 which is incompatible.


In [ ]:
!pip install pypdf
!pip install python-dotenv
!pip install -q transformers einops accelerate langchain bitsandbytes
!pip install sentence_transformers
!pip install llama-index



In [ ]:
# !pip install googletrans==4.0.0-rc1

# from googletrans import Translator

In [ ]:
import logging
import sys
# import subprocess
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms import HuggingFaceLLM

In [ ]:
documents = SimpleDirectoryReader("/content/content").load_data()

In [ ]:

from llama_index.prompts.prompts import SimpleInputPrompt


system_prompt = "you are a legal aid for an Indian citizen.Help the user know the right answer for their queries in the most precise way possible. the answer should be abide by the laws of democracy and Constitution of India.The answer should be so apt and legally correct and all the articles, recent amendments  and consider the legal history in a very wise way.Contemplate before answering. Provide a lengthy friendly yet strict response going into details of the case.response should include arcticles and sections related to the issue. Answer should be in html format using div and other html tags and highlight important advices in bold."



# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 

In [ ]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=1024,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)

In [ ]:
# # from llama_index import LangchainEmbedding
# from llama_index.embedings import LangchainEmbedding

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings


embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")



In [ ]:
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)


In [ ]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [ ]:
  query_engine = index.as_query_engine()

In [ ]:
while True:
    query = input("Enter your query (or 'exit' to quit): ")
    if query == "exit":
        break

    # Pass the user's query to your search engine
    response = query_engine.query(query)  # Assuming 'index' is your search engine
    response_text = response.response

    # Print the response
    print("Response:", response_text)
